# Explore here

In [26]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [27]:
df = pd.read_csv('../data/raw/url_spam.csv')
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [28]:
# Preprocesamiento de las URLs
def procesar_url(texto):
    # Minúsculas y separación por caracteres no alfabéticos
    tokens = re.split(r'\W+', texto.lower())
    tokens = [t for t in tokens if len(t) > 2]  # eliminar tokens muy cortos
    return " ".join(tokens)

df["url_limpia"] = df["url"].apply(procesar_url)

Procesamos las URLs como si fueran cadenas complejas. Extraemos tokens de la URL separándola por tokens.

In [29]:
# Dividimos el dataset.
X = df["url_limpia"]
y = df["is_spam"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Separamos los datos en dos grupos: unos para entrenar el modelo y otros para comprobar si funciona bien.

In [30]:
# Vectorizamos con TFIDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

El modelo no entiende texto, así que usamos TF-IDF para transformar las URLs procesadas en vectores numéricos.

In [31]:
# SVM para entrenar el modelo.
modelo = SVC(kernel="linear", random_state=42)
modelo.fit(X_train_tfidf, y_train)

y_pred = modelo.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.96      0.96       455
        True       0.88      0.90      0.89       145

    accuracy                           0.94       600
   macro avg       0.92      0.93      0.93       600
weighted avg       0.95      0.94      0.95       600



Precision (Spam: 0.88): Cuando el modelo dice que algo es spam, acierta el 88% de las veces. Un 12% de falsos positivos.

Recall (Spam: 0.90): Detecta el 90% de los verdaderos enlaces spam.

F1-score (Spam: 0.89): Es el promedio equilibrado entre precisión y recall. Parece ser sólido.

In [32]:
# Optimizar el modelo
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid = GridSearchCV(SVC(), param_grid, scoring='f1', cv=5)
grid.fit(X_train_tfidf, y_train)

print("Mejores parámetros:", grid.best_params_)
print("Mejor score:", grid.best_score_)

Mejores parámetros: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Mejor score: 0.9258462350692657


C: 10 → más penalización a errores (modelo más exigente).

kernel: 'linear' → indica que los datos son bastante separables linealmente.

gamma: 'scale' → aunque no afecta directamente al kernel lineal, fue evaluado igual.

Mejor F1-score (validación cruzada): 0.926, el modelo tiene una excelente capacidad de equilibrio entre precisión y recall en distintas particiones de los datos.